In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm.notebook import tqdm
from statistics import mean

import torchvision.transforms as transforms
import torch
import torch.nn.functional as F

from models.old_model import SiameseNetwork
from models.new_model import NewSiameseNetwork
from models import pretrained_model

from utils.other_utils import joinpath, TestNetworkDataset, get_dataset
from utils.other_utils import intersection, threshold

from embeddings.create_embeddings import create_embeddings
from embeddings.mean_embeddings import mean_embeddings

from scoring.test_report import test_report

In [3]:
# # Setup the path where images are located
# PATH = "/home/akshatgarg/Just-In-Time-Detection-of-Unknown-Classes-Using-Deep-Learning/data/brake shoe"

# # # save the model path
# MODEL_PATH = "/home/akshatgarg/Just-In-Time-Detection-of-Unknown-Classes-Using-Deep-Learning/saved_model/brake_shoe_model"

PATH = "data/lab_parts"

# # save the model path
MODEL_PATH = "saved_model/lab_parts_100_64_04_26_2023_02_52"

# Setup the number of channels in the image, 3 for color, 1 for monochrome
nchannel=3


# whether it is triplet or contrastive
FLAG = 'contrastive'
# FLAG = 'triplet'

# rotation flag
ROTATION = False

# if testing
testing = False

# Threshold score
THRESHOLD_SCORE = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(torch.cuda.get_device_name())
    torch.cuda.empty_cache()

NVIDIA GeForce RTX 3070 Laptop GPU


In [4]:
# Resize the images and transform to tensors
if ROTATION:
    transformation = transforms.Compose([transforms.Resize((224,224)),
        transforms.ToTensor(), 
        transforms.RandomApply([
            transforms.RandomRotation([-30,30])], p = 0.2)
    ])
else:
    # transformation = transforms.Compose([transforms.Resize((100,100)),
    #                                  transforms.ToTensor()])
    transformation = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])] 
                                     )

In [5]:
# Getiing the saved model from MODEL_PATH

net = pretrained_model.SiameseNetwork().to(device)  # create an instance of your model class
net.load_state_dict(torch.load(joinpath(MODEL_PATH,'model.pt')))
net.eval()

/home/akshatgarg/anaconda3/envs/torchcapstone/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/akshatgarg/anaconda3/envs/torchcapstone/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SiameseNetwork(
  (resnet18): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [5]:
# # Save the trained model using pickle
# with open("trained_model.pkl", "wb") as f:
#     pickle.dump(net, f)

# # Load the saved model using pickle
# with open("trained_model.pkl", "rb") as f:
#     loaded_model = pickle.load(f)

# loaded_model.eval()  # Set

In [6]:
# #### here we create dataiter for test

# # Locate the test dataset and load it into the SiameseNetworkDataset_for_test
# if testing:
#     test_dataloader,_ = get_dataset(joinpath(PATH,'test'), nchannel, transformation, TestNetworkDataset, FLAG, 0, 1, True)

#     # Grab one image that we are going to test
#     dataiter = iter(test_dataloader)
#     x0,_ = next(dataiter)


#     #### here we create dataiter for train.

#     # Locate the test dataset and load it into the SiameseNetworkDataset_for_test
#     train_dataloader,siamese_dataset_train = get_dataset(joinpath(PATH,'train'), nchannel, transformation, TestNetworkDataset, FLAG, 0, 1, False)

#     dataiter_train = iter(train_dataloader)

#     score = []

#     for i in range(len(siamese_dataset_train)):
#         # Iterate over all images and test them with the first image (x0)
#         x1,_ = next(dataiter_train)

#         # Concatenate the two images together
#         concatenated = torch.cat((x0, x1), 0)
        
#         output1, output2 = net(x0.cto(device), x1.to(device))
#         euclidean_distance = F.pairwise_distance(output1, output2)
#         score.append(round(euclidean_distance.item(),2))
#         imshow(torchvision.utils.make_grid(concatenated), f'Dissimilarity: {euclidean_distance.item():.2f}')
#         # cosine_distance = torch.cosine_similarity(output1, output2)
#         # score.append(round(cosine_distance.item(),2))
#         # imshow(torchvision.utils.make_grid(concatenated), f'Dissimilarity: {cosine_distance.item():.2f}')

In [7]:
#### here we create dataiter for test

test_dataloader,siamese_dataset = get_dataset(joinpath(PATH,'test'), nchannel, transformation, TestNetworkDataset, FLAG, 0, 1, True)

#### here we create dataiter for train

train_dataloader,siamese_dataset_train = get_dataset(joinpath(PATH,'train'), nchannel, transformation, TestNetworkDataset, FLAG, 0, 1, False)


truth=[]
predict=[]
same_disssimilarity=[]
different_disssimilarity=[]


dataiter = iter(test_dataloader)
for i in tqdm(range(len(siamese_dataset)),desc='TestCount'):
    
    x0,label0 = next(dataiter)
    # count_label0.append(label0)
    

# Grab one image that we are going to test
    dataiter_train = iter(train_dataloader)

    for i in range(len(siamese_dataset_train)):
    # Iterate over 5 images and test them with the first image (x0)
        x1,label1 = next(dataiter_train)
        # count_label1.append(label1)

        # Concatenate the two images together
        concatenated = torch.cat((x0, x1), 0)

        output1, output2 = net(x0.to(device), x1.to(device))
        euclidean_distance = F.pairwise_distance(output1, output2)
        

        predict.append(euclidean_distance.item())
        if label0==label1: # if labels of two images are same append them to same_dissimilar
            same_disssimilarity.append(euclidean_distance.item())
        else:
            different_disssimilarity.append(euclidean_distance.item())

        # cosine_distance = torch.cosine_similarity(output1, output2)
        # predict.append(cosine_distance.item())
        # if label0==label1:
        #     same_disssimilarity.append(cosine_distance.item())
        # else:
        #     different_disssimilarity.append(cosine_distance.item())

TestCount:   0%|          | 0/438 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# checking if there are any intersection in same and different images
print("Number of total interesection: ", len(intersection(same_disssimilarity,different_disssimilarity)))

In [ ]:
import pickle

# open a file in binary write mode
with open(joinpath(MODEL_PATH,'disssimilarity_lists.pkl'), 'wb') as f:
    # pickle the two lists
    pickle.dump(same_disssimilarity, f)
    pickle.dump(different_disssimilarity, f)


In [ ]:
import pickle

# open the file in binary read mode
with open(joinpath(MODEL_PATH,'disssimilarity_lists.pkl'), 'rb') as f:
    # unpickle the two lists
    same_disssimilarity = pickle.load(f)
    different_disssimilarity = pickle.load(f)

In [ ]:
# Create figure and axes objects
fig, ax = plt.subplots(figsize=(8, 6))

# Set custom color palette
colors = ["#E74C3C", "#3498DB"]

# Set background color
ax.set_facecolor("#F7F7F7")

# Plot the distributions and set custom colors
sns.kdeplot(same_disssimilarity, bw_method=0.5, label="Same", color=colors[0])
sns.kdeplot(different_disssimilarity, bw_method=0.5, label="Different", color=colors[1])

# Add legend and title
plt.legend(fontsize=14)
plt.title('Density vs Dissimilarity for euclidean distance', fontsize=20)

# Add gridlines
ax.grid(linestyle='--', linewidth=0.5)

# Set axis labels and tick sizes
plt.xlabel("Dissimilarity", fontsize=16)
plt.ylabel("Density", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Remove top and right spines
sns.despine(top=True, right=True)

plt.show()


In [ ]:
import plotly.figure_factory as ff
import numpy as np

x1 = same_disssimilarity
x2 = different_disssimilarity

hist_data = [x1, x2]

group_labels = ["Same", "Different"]
# colors = ['#333F44', '#37AA9C']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.15, show_hist=True, curve_type='kde')

# Add title
fig.update_layout(title_text='Curve and Rug Plot')
fig.show()

In [ ]:
# checking for mean/average score
print("Mean of same images: ",mean(same_disssimilarity))
print("Mean of different images: ",mean(different_disssimilarity))

In [ ]:
# Prinitng total number of combinations that we tested on
print("Total number of combinations of same images: ", len(same_disssimilarity))
print("Total number of combinations of different images: ", len(different_disssimilarity))
print("Total number of combinations: ", len(same_disssimilarity) + len(different_disssimilarity))

In [ ]:
# Checking for number of missclassified images
final_list, same_misclassified, different_misclassified = threshold(same_disssimilarity,different_disssimilarity,THRESHOLD_SCORE)

print("Total number of missclassified for threshold score ",THRESHOLD_SCORE," missclassified images: ",len(final_list))
print("Total number of same class missclassified for threshold score ",THRESHOLD_SCORE," missclassified images: ",len(same_misclassified))
print("Total number of different class missclassified for threshold score ",THRESHOLD_SCORE," missclassified images: ",len(different_misclassified))

In [6]:
embedding_dataloader, embedding_dataset = get_dataset(joinpath(PATH,'train'), nchannel, transformation, TestNetworkDataset, FLAG, 0, 1, False)

dict_embeddings = create_embeddings(net, device, embedding_dataloader, embedding_dataset)
dict_mean = mean_embeddings(dict_embeddings)

In [7]:
from embeddings.extract_labels_embeddings import extract_labels_and_embeddings
embeddings, labels = extract_labels_and_embeddings(dict_embeddings)
# embeddings

In [8]:
test_dataloader, test_dataset = get_dataset(joinpath(PATH,'test'), nchannel, transformation, 
                                            TestNetworkDataset, FLAG, 0, 1, False)

report, execution_time, avg_time_image = test_report(PATH, net, device, test_dataloader, test_dataset, dict_embeddings, THRESHOLD_SCORE, 'base', 10)

  0%|          | 0/438 [00:00<?, ?it/s]

In [9]:
print(report)
print("Execution time per image: ", mean(avg_time_image), " seconds")
print("Execution time: ", execution_time, " seconds")

                      precision    recall  f1-score   support

   3176 CAT Injector       1.00      1.00      1.00        29
               4515Q       1.00      1.00      1.00        50
               4524Q       1.00      1.00      1.00        50
               4707Q       0.50      1.00      0.67        50
               4725E       1.00      1.00      1.00        50
               4729E       1.00      1.00      1.00        50
           6.7L Ford       0.57      1.00      0.72        25
7.3L Ford + 3126 CAT       1.00      1.00      1.00        26
           Bosch LML       0.89      1.00      0.94        32
       Unknown Class       1.00      0.04      0.08        76

            accuracy                           0.83       438
           macro avg       0.90      0.90      0.84       438
        weighted avg       0.91      0.83      0.78       438

Execution time per image:  0.04961139045349539  seconds
Execution time:  21.874744653701782  seconds


In [10]:
test_dataloader, test_dataset = get_dataset(joinpath(PATH,'test'), nchannel, transformation, 
                                            TestNetworkDataset, FLAG, 0, 1, False)

report, execution_time, avg_time_image = test_report(PATH, net, device, test_dataloader, test_dataset, dict_embeddings, THRESHOLD_SCORE, 'all', 10)

  0%|          | 0/438 [00:00<?, ?it/s]

In [11]:
print(report)
print("Execution time per image: ", mean(avg_time_image), " seconds")
print("Execution time: ", execution_time, " seconds")

                      precision    recall  f1-score   support

   3176 CAT Injector       1.00      1.00      1.00        29
               4515Q       1.00      1.00      1.00        50
               4524Q       1.00      1.00      1.00        50
               4707Q       0.50      1.00      0.67        50
               4725E       1.00      1.00      1.00        50
               4729E       1.00      1.00      1.00        50
           6.7L Ford       0.57      1.00      0.72        25
7.3L Ford + 3126 CAT       1.00      1.00      1.00        26
           Bosch LML       0.89      1.00      0.94        32
       Unknown Class       1.00      0.04      0.08        76

            accuracy                           0.83       438
           macro avg       0.90      0.90      0.84       438
        weighted avg       0.91      0.83      0.78       438

Execution time per image:  0.05676990393634256  seconds
Execution time:  25.022639751434326  seconds


In [12]:
# test_dataloader, test_dataset = get_dataset(joinpath(PATH,'test'), transformation, TestNetworkDataset, FLAG, 0, 1, False)
# dataiter = iter(test_dataloader)
# image, label = next(dataiter)

# print(label[0])
# n_way_dict = n_way_shot_learning(net, device, image, dict_mean, THRESHOLD_SCORE)
# print(n_way_dict)

In [13]:
my_list = [round(x * 0.01, 2) for x in range(-300, 301)]
final = []
same = []
different = []
for i in my_list:
    a,b,c = threshold(same_disssimilarity,different_disssimilarity,i)
    final.append(len(a))
    same.append(len(b))
    different.append(len(c))

NameError: name 'same_disssimilarity' is not defined

In [ ]:
import plotly.express as px

# create the line plot
fig = px.line(x=my_list, y=final)

# set the x-axis range to -1 to 1 and y-axis range to -100 to 2500
fig.update_xaxes(range=[-1, 2])
fig.update_yaxes(range=[-100, 5000])

# add a vertical line at x=0.2
fig.add_shape(type='line', x0=0.2, x1=0.2,
              y0=0, y1=max(final),
              line=dict(color='red', width=1, dash='dash'),
              name='Minimum Number of Misclassification')

fig.update_layout(
    title={
        'text': 'Total missclassification vs threshold',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Threshold',
    yaxis_title='Missclassified images',
    xaxis=dict(showgrid=True, gridwidth=1, gridcolor='DarkGray', linecolor='DarkGray', linewidth=2),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='DarkGray', linecolor='DarkGray', linewidth=2),
    showlegend=True
)
# display the plot
fig.show()

In [ ]:
import plotly.graph_objects as go

# create trace for the first line
trace1 = go.Scatter(x=my_list, y=same, mode='lines', name='Same class Images')

# create trace for the second line
trace2 = go.Scatter(x=my_list, y=different, mode='lines', name='Different class Images')

# create the figure object and add both traces
fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)

# set the x-axis range to -1 to 1 and y-axis range to -100 to 2500
fig.update_xaxes(range=[-0.5, 1.5])
fig.update_yaxes(range=[-100, 1500])

# find the intersection point of the two lines
intersection_point = my_list[same.index(max(same))]

# add a vertical line at x=0.2
fig.add_shape(type='line', x0=0.19, x1=0.19,
              y0=0, y1=max(final),
              line=dict(color='red', width=1, dash='dash'),
              name='Intersection of both')


fig.update_layout(
    title={
        'text': 'Optimal Threshold Value for Minimum Misclassification',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Threshold',
    yaxis_title='Missclassified images',
    xaxis=dict(showgrid=True, gridwidth=1, gridcolor='DarkGray', linecolor='DarkGray', linewidth=2),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='DarkGray', linecolor='DarkGray', linewidth=2),
    showlegend=True
)

# display the plot
fig.show()


In [14]:
test_dataloader, test_dataset = get_dataset(joinpath(PATH,'test'), nchannel, transformation, 
                                            TestNetworkDataset, FLAG, 0, 1, False)

report, execution_time, avg_time_image = test_report(PATH, net, device, test_dataloader, test_dataset, dict_mean, 0.5, 'mean', 10)

  0%|          | 0/438 [00:00<?, ?it/s]

/home/akshatgarg/Just-In-Time-Detection-of-Unknown-Classes-Using-Deep-Learning/embeddings/n_way_shot_learning_mean.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  values_softmax = F.softmax(values)


In [15]:
print(report)
print("Execution time per image: ", mean(avg_time_image), " seconds")
print("Execution time: ", execution_time, " seconds")

                      precision    recall  f1-score   support

   3176 CAT Injector       1.00      0.97      0.98        29
               4515Q       1.00      0.78      0.88        50
               4524Q       1.00      1.00      1.00        50
               4707Q       1.00      1.00      1.00        50
               4725E       1.00      0.96      0.98        50
               4729E       1.00      1.00      1.00        50
           6.7L Ford       1.00      1.00      1.00        25
7.3L Ford + 3126 CAT       1.00      1.00      1.00        26
           Bosch LML       0.96      0.84      0.90        32
       Unknown Class       0.80      0.99      0.88        76

            accuracy                           0.95       438
           macro avg       0.98      0.95      0.96       438
        weighted avg       0.96      0.95      0.95       438

Execution time per image:  0.03559390268369352  seconds
Execution time:  15.693552255630493  seconds
